In [ ]:
from google.colab import drive
import ee
import geemap
import pandas as pd
import numpy as np
import random
import os
import cv2 as cv


drive.mount('/content/drive')

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='andres-gara2024')

In [ ]:
def sar_imageries_mean(start, end, latitude, longitude, polarization, direction='DESCENDING', degrees=0.1):
    degrees2 = degrees / 2
    p11 = ee.Geometry.Point(longitude, latitude)
    p12 = ee.Geometry.Point(longitude, latitude + degrees)
    p21 = ee.Geometry.Point(longitude + degrees2, latitude)
    p22 = ee.Geometry.Point(longitude + degrees2, latitude + degrees)

    sar = (
        ee.ImageCollection('COPERNICUS/S1_GRD')
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization))
        .filterBounds(p11)
        .filterBounds(p12)
        .filterBounds(p21)
        .filterBounds(p22)
        .filterDate(start, end)
        .filter(ee.Filter.eq('instrumentMode', 'IW'))
        .filter(ee.Filter.eq('orbitProperties_pass', direction))
        .select(polarization)
    )

    roi = ee.Geometry.Rectangle(longitude, latitude, longitude + degrees, latitude + degrees / 2)
    sar_roi = sar.mean().clip(roi)

    return sar_roi


def optics_imageries_mean(start, end, latitude, longitude, cloudiness=5, water_percentage=40, snow_percentage=5, vegetation_percentage=10, degrees=0.1):
    degrees2 = degrees / 2
    p11 = ee.Geometry.Point(longitude, latitude)
    p12 = ee.Geometry.Point(longitude, latitude + degrees)
    p21 = ee.Geometry.Point(longitude + degrees2, latitude)
    p22 = ee.Geometry.Point(longitude + degrees2, latitude + degrees)

    optics = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterDate(start, end)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloudiness))
        .filter(ee.Filter.lt('SNOW_ICE_PERCENTAGE', snow_percentage))
        .filter(ee.Filter.lt('WATER_PERCENTAGE', water_percentage))
        .filter(ee.Filter.gt('VEGETATION_PERCENTAGE', vegetation_percentage))
        .filterBounds(p11)
        .filterBounds(p12)
        .filterBounds(p21)
        .filterBounds(p22)
    )

    roi = ee.Geometry.Rectangle(longitude, latitude, longitude + degrees, latitude + degrees / 2)
    optics_roi = optics.mean().clip(roi)
    size_value = optics.size().getInfo()

    if size_value > 0:
        optics_list = optics.toList(optics.size())

        cloudiness_volatil = []
        water_percentage_volatil = []
        vegetation_percentage_volatil = []

        for i in range(optics_list.size().getInfo()):
            img = optics.toList(optics_list.size()).get(i)
            img = ee.Image(img)
            img = img.clip(roi)

            propiedades = img.getInfo()
            cloudiness_value = propiedades['properties']['CLOUD_COVERAGE_ASSESSMENT']
            water_percentage_value = propiedades['properties']['WATER_PERCENTAGE']
            vegetation_percentage_value = propiedades['properties']['VEGETATION_PERCENTAGE']

            cloudiness_volatil.append(cloudiness_value)
            water_percentage_volatil.append(water_percentage_value)
            vegetation_percentage_volatil.append(vegetation_percentage_value)

        cloudiness_mean_volatil = sum(cloudiness_volatil) / len(cloudiness_volatil)
        water_percentage_mean_volatil = sum(water_percentage_volatil) / len(water_percentage_volatil)
        vegetation_percentage_mean_volatil = sum(vegetation_percentage_volatil) / len(vegetation_percentage_volatil)

        optics_roi = optics.mean().clip(roi)
        return size_value, optics_roi, cloudiness_mean_volatil, water_percentage_mean_volatil, vegetation_percentage_mean_volatil

    return size_value, optics_roi, 0, 0, 0


In [ ]:
folder_sar = '/content/drive/MyDrive/Main_Folder/Original/SAR/'
folder_ndvi = '/content/drive/MyDrive/Main_Folder/Original/NDVI/'
folder_optics = '/content/drive/MyDrive/Main_Folder/Original/Optics/'
metadata = '/content/drive/MyDrive/Main_Folder/Metadata/'

os.makedirs(folder_sar + 'VV', exist_ok=True)
os.makedirs(folder_sar + 'VH', exist_ok=True)
os.makedirs(folder_ndvi, exist_ok=True)
os.makedirs(folder_optics, exist_ok=True)
os.makedirs(metadata, exist_ok=True)


In [ ]:
i = 0
number_images = 5

initial_dates = []
end_dates = []
latitudes = []
longitudes = []
cloudiness_mean = []
water_percentage_mean = []
vegetation_percentage_mean = []

while i < number_images:

    latitude = random.uniform(-40, 40)
    longitude = random.uniform(-179, 179)
    day = random.randint(1, 28)
    month = random.randint(1, 9)
    year = random.randint(17, 23)

    start_date = '20' + str(year) + '-' + str(month) + '-' + str(day)
    end_date = '20' + str(year) + '-' + str(month + 3) + '-' + str(day)

    sar_vv = sar_imageries_mean(start_date, end_date, latitude, longitude, polarization='VV')
    sar_vh = sar_imageries_mean(start_date, end_date, latitude, longitude, polarization='VH')
    size_value, optics, cloudiness_mean_volatil, water_percentage_mean_volatil, vegetation_percentage_mean_volatil = optics_imageries_mean(
        start_date, end_date, latitude, longitude)

    if size_value > 0:
        size_value, optics, cloudiness_mean_volatil, water_percentage_mean_volatil, vegetation_percentage_mean_volatil = optics_imageries_mean(
            start_date, end_date, latitude, longitude)
        condition_1 = cloudiness_mean_volatil < 5
        condition_2 = water_percentage_mean_volatil < 40
        condition_3 = vegetation_percentage_mean_volatil > 10

        if (len(sar_vv.getInfo()['bands']) and len(optics.getInfo()['bands']) and condition_1 and condition_2 and condition_3):

            ndvi = optics.normalizedDifference(['B8', 'B4'])
            optics_vis = optics.visualize(**{'min': 0.0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']})
            sar_vv_vis = sar_vv.visualize(**{'min': -25, 'max': 5})
            sar_vh_vis = sar_vh.visualize(**{'min': -25, 'max': 5})

            geemap.ee_export_image(ndvi, filename=folder_ndvi + str(i) + '.tif', scale=10, file_per_band=False)
            geemap.ee_export_image(optics_vis, filename=folder_optics + str(i) + '.tif', scale=10, file_per_band=False)
            geemap.ee_export_image(sar_vv_vis, filename=folder_sar + 'VV/' + str(i) + '.tif', scale=10, file_per_band=False)
            geemap.ee_export_image(sar_vh_vis, filename=folder_sar + 'VH/' + str(i) + '.tif', scale=10, file_per_band=False)

            initial_dates.append(start_date)
            end_dates.append(end_date)
            latitudes.append(latitude)
            longitudes.append(longitude)

            cloudiness_mean.append(cloudiness_mean_volatil)
            water_percentage_mean.append(water_percentage_mean_volatil)
            vegetation_percentage_mean.append(vegetation_percentage_mean_volatil)

            print(f'Imagery #({i}) saved successfully!')
            i += 1


In [ ]:
metadata_xlsx = '/content/drive/MyDrive/Main_Folder/Metadata/Data.xlsx'
metadata_csv = '/content/drive/MyDrive/Main_Folder/Metadata/Data.csv'

df = pd.DataFrame(data={
    'Initial dates': initial_dates,
    'End dates': end_dates,
    'Latitude': latitudes,
    'Longitude': longitudes,
    'Cloudiness Mean': cloudiness_mean,
    'Water Percentage Mean': water_percentage_mean,
    'Vegetation Percentage Mean': vegetation_percentage_mean
})

df.to_csv(metadata_csv, index=False)
df.to_excel(metadata_xlsx, index=False)

In [ ]:
input_images_path = '/content/drive/MyDrive/Main_Folder/Original/'
output_images_path = '/content/drive/MyDrive/Main_Folder/Crop/'
paths = ['SAR/VH/', 'Optics/', 'NDVI/', 'SAR/VV/']

for path in paths:
    files_names = os.listdir(input_images_path + path)
    os.makedirs(output_images_path + path, exist_ok=True)

    for file_name in files_names:
        image_path = os.path.join(input_images_path, path, file_name)

        img2 = cv.imread(image_path, cv.IMREAD_UNCHANGED)
        img3 = img2[:512, :512]  # Top-left crop
        img4 = img2[:512, 512:1024]  # Top-right crop

        split = file_name.split('.')
        cv.imwrite(os.path.join(output_images_path, path, f"{split[0]}A.{split[1]}"), img3)
        cv.imwrite(os.path.join(output_images_path, path, f"{split[0]}B.{split[1]}"), img4)

    print(f'Finished {path} folder')